In [ ]:
import xarray as xr
import geopandas as gpd
import numpy as np

def bridge_vulnerability_temperature(temperature_data, temperature_variable):
    #Define threshold temperatures
    temperature_thresholds = {
        'low': 16.0,   # Below this temperature, minimal stress on the bridge
        'high': 30.0,  # Above this temperature, increased stress on the bridge
        'threshold_exponential': 35.0,  # Temperature threshold for exponential vulnerability increase
    }
    temperature = temperature_data[temperature_variable]

    #Vulnerability scores
    if temperature < temperature_thresholds['low']:
        vulnerability_score = 1.0
    elif temperature >= temperature_thresholds['low'] and temperature < temperature_thresholds['high']:
        # Linear interpolation between low and high thresholds
        vulnerability_score = 1.0 - ((temperature - temperature_thresholds['low']) / (temperature_thresholds['high'] - temperature_thresholds['low']))
    elif temperature >= temperature_thresholds['high'] and temperature < temperature_thresholds['threshold_exponential']:
        vulnerability_score = 0.0  # Maximum stress on the bridge
    else:
        # Exponential increase in vulnerability score after the exponential threshold
        vulnerability_score = np.exp(1.0 - (temperature - temperature_thresholds['threshold_exponential']))

    return vulnerability_score

temperature_variable = '_osc_chronic_heat_prefix()'
temperature = climate_data[temperature_variable]

vulnerability_scores = np.zeros_like(temperature)  #Array to store vulnerability scores

for i in range(temperature.shape[0]):
    for j in range(temperature.shape[1]):
        temperature_value = temperature[i, j].item()
        vulnerability_score = bridge_vulnerability_temperature(temperature_data=temperature_value, temperature_variable=temperature_variable)
        vulnerability_scores[i, j] = vulnerability_score

#GeoDataFrame for vulnerability scores and coordinates
lon, lat = climate_data.longitude.values, climate_data.latitude.values
vulnerability_df = gpd.GeoDataFrame({'vulnerability_score': vulnerability_scores.flatten()}, geometry=gpd.points_from_xy(lon, lat))

#Export vulnerability scores
vulnerability_df.to_file('bridge_vulnerability.geojson', driver='GeoJSON')
print("Bridge vulnerability scores exported as GeoJSON.")
